In [113]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import explained_variance_score


dataFrame = pd.read_csv("NFLDraftDataCleaned.csv")

In [114]:
#Create new dataframes to hold independent and dependent variables
# X = dataFrame[['position', 'age', 'height', 'weight', 'collegeAttended', 'rushingAttempts', 'rushingYards',
#                            'avgYardsPerRush', 'rushingTouchdowns', 'receptions', 'receivingYards', 'avgYardsPerReception',
#                            'receivingTouchdowns', 'fortyYardDash', 'verticalJump', 'benchPress', 'broadJump', 'threeCone',
#                            'shuttle']]
X = dataFrame[['age', 'height', 'weight', 'rushingAttempts', 'rushingYards',
                           'avgYardsPerRush', 'rushingTouchdowns', 'receptions', 'receivingYards', 'avgYardsPerReception',
                           'receivingTouchdowns', 'fortyYardDash', 'verticalJump', 'benchPress', 'broadJump', 'threeCone',
                           'shuttle']]
y = dataFrame[['draftRound']]

In [115]:
# #Some of the dependent variables (position and collegeAttended) are stored as categorical data
# #This needs to be converted to numerical data using OneHotEncoder
# onehot = pd.get_dummies(X['position'])
# X = X.drop('position', axis = 1)
# X = X.join(onehot)

# onehot = pd.get_dummies(X['collegeAttended'])
# X = X.drop('collegeAttended', axis = 1)
# X = X.join(onehot)

In [116]:
#Split the dataset into some training data and testing data
X_Train, X_Test, y_Train, y_Test = train_test_split(X, y, test_size=0.2)

#Create a linear regression model and fit it to the train data
linearModel = LinearRegression()
linearModel.fit(X_Train, y_Train)

#Predict the draft round for the test set
unformattedPredictions = linearModel.predict(X_Test)

predictions = []

#There are some errors with our predictions, they are giving wildly higher scores
for prediction in unformattedPredictions:
    if(prediction < 1.5):
        predictions.append(1)
    if(prediction >= 1.5 and prediction < 2.5):
        predictions.append(2)
    if(prediction >= 2.5 and prediction < 3.5):
        predictions.append(3)
    if(prediction >= 3.5 and prediction < 4.5):
        predictions.append(4)
    if(prediction >= 4.5 and prediction < 5.5):
        predictions.append(5)
    if(prediction >= 5.5 and prediction < 6.5):
        predictions.append(6)
    if(prediction >= 6.5):
        predictions.append(7)

print(unformattedPredictions)    

explainedVarianceScore = explained_variance_score(y_Test, predictions)
print("Explained variance score = ", explainedVarianceScore)

comparisonFrame = pd.DataFrame({'Actual': y_Test['draftRound'], 'predicted': predictions})
comparisonFrame

[[3.58351207]
 [4.58878342]
 [3.15894949]
 [3.57460007]
 [3.50309831]
 [4.16442518]
 [3.93373041]
 [2.52139051]
 [3.16488215]
 [4.49251674]
 [4.06701045]
 [4.21040782]
 [3.56424399]
 [4.6958171 ]
 [3.66968213]
 [1.54363699]
 [4.46428404]
 [2.72843465]
 [4.39570076]
 [4.1368333 ]
 [4.40746393]
 [3.7321543 ]
 [3.56462534]
 [4.22276892]
 [2.88536514]
 [3.51576567]
 [5.41221305]
 [6.46041965]
 [2.11035499]
 [3.29857798]
 [2.16767077]
 [2.38055512]
 [6.29911843]
 [5.20727554]
 [3.17402483]
 [5.01661655]
 [5.03339774]
 [3.25860233]
 [3.76372458]
 [5.05557909]
 [4.84922086]
 [5.04751204]
 [2.77393015]
 [5.08632188]
 [4.69239419]
 [3.40031936]
 [2.47958794]
 [2.80383967]
 [4.51863052]
 [3.07977314]
 [3.87964046]
 [3.80516894]
 [3.96604211]
 [5.05856064]
 [3.17659906]
 [4.20423609]
 [2.56081099]
 [3.97476621]
 [4.77376989]
 [5.20712064]
 [4.61967116]
 [3.44530397]
 [3.92328255]
 [3.25360589]
 [4.98542122]
 [3.16454436]
 [3.06745737]
 [4.64660214]
 [3.71704874]
 [5.14120583]
 [2.89394854]
 [5.07

,Actual,predicted
512,5,4
663,4,5
837,7,3
653,3,4
561,5,4
508,5,4
13,2,4
646,2,3
849,2,3
501,4,4
